In [1]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
import re

In [2]:
df = pd.read_csv('../MachineHackElectronicPrice/Train.csv')
df_aggp = pd.read_csv('../MachineHackElectronicPrice/train_AGGp.csv')
df_nlp = pd.read_csv('../MachineHackElectronicPrice/train_NLP.csv')
df_topic = pd.read_csv('../MachineHackElectronicPrice/train_topic.csv')
df_embed = pd.read_csv('../MachineHackElectronicPrice/train_embedding.csv')

In [3]:
tf = pd.read_csv('../MachineHackElectronicPrice/Test.csv')
tf_aggp = pd.read_csv('../MachineHackElectronicPrice/test_AGGp.csv')
tf_nlp = pd.read_csv('../MachineHackElectronicPrice/test_NLP.csv')
tf_topic = pd.read_csv('../MachineHackElectronicPrice/test_topic.csv')
tf_embed = pd.read_csv('../MachineHackElectronicPrice/test_embedding.csv')

In [4]:
df = df.drop(['Model_Info','Additional_Description'] ,axis =1)
tf = tf.drop(['Model_Info','Additional_Description'] ,axis =1)

In [32]:
data = pd.concat([df,df_aggp,df_nlp,df_topic,df_embed],axis=1)
test = pd.concat([tf,tf_aggp,tf_nlp,tf_topic,tf_embed],axis=1)

# model time

In [6]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [8]:
y_train = data.Price

train= data.drop(['Price'],axis=1)

In [9]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmae= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_absolute_error", cv = kf))
    return(rmae)

In [10]:
GBoost = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [11]:
# model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=720,
#                               max_bin = 55, bagging_fraction = 0.8,
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
model_lgb= lgb.LGBMRegressor(objective='regression',n_estimators=150, learning_rate=0.09, num_leaves=42, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

In [12]:
model_rf = RandomForestRegressor(n_estimators=100,max_features ='auto',)
from sklearn.model_selection import GridSearchCV


In [13]:
# param_grid = { 
#     'n_estimators': [200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
#     'criterion' :['gini', 'entropy']
# }

In [14]:
# CV_rfc = GridSearchCV(estimator=model_rf, param_grid=param_grid, cv= 5)
# CV_rfc.fit(train,y_train)

In [15]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


In [181]:
# CV_rfc = GridSearchCV(estimator=model_rf, param_grid=random_grid, cv= 3)
# CV_rfc.fit(train,y_train)

In [182]:
# score = rmsle_cv(GBoost)
# print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [183]:
# score = rmsle_cv(model_lgb)
# print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [184]:
# score = rmsle_cv(model_rf)
# print("RF score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [16]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [17]:
#averaged_models = AveragingModels(models = ( GBoost,model_rf))

averaged_models = AveragingModels(models = ( GBoost,model_rf,model_lgb))
#### withlog

In [187]:
# averaged_models.fit(train.values,np.log1p(y_train))

In [188]:
# avg_pred = np.expm1(averaged_models.predict(test.values))


In [189]:
# sub = pd.DataFrame()
# sub['gift_id'] = test_ID
# sub['price'] = avg_pred
# sub.to_csv('submissionlog2.csv',index=False)

In [190]:
# model_rf.fit(train,np.log1p(y_train))
# pred = np.expm1(model_rf.predict(test))

In [191]:
# sub = pd.DataFrame()
# sub['gift_id'] = test_ID
# sub['price'] = pred
# sub.to_csv('submissionRF.csv',index=False)

# log chal gya 

In [192]:
#y_train = np.log1p(y_train)

In [18]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [19]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [20]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [21]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet,GBoost),
                                                 meta_model = lasso)

# score = rmsle_cv(stacked_averaged_models)
# print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [22]:
stacked_averaged_models.fit(train, np.log1p(y_train))
stacked_averaged_models_train_pred = np.expm1(stacked_averaged_models.predict(train))

stacked_averaged_models_pred = np.expm1(stacked_averaged_models.predict(test))
print(rmsle(y_train, stacked_averaged_models_train_pred))

KeyError: "None of [Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,\n            ...\n            2310, 2311, 2313, 2314, 2315, 2320, 2322, 2323, 2324, 2325],\n           dtype='int64', length=1860)] are in the [columns]"

In [23]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import StackingRegressor

In [24]:
# estimators = [
#      ('rf', model_rf)]

In [25]:
# reg = StackingRegressor(
#      estimators=estimators,
#      final_estimator= model_rf)

In [26]:
# reg.fit(train.values, np.log1p(y_train))

In [27]:
# reg_pred = np.expm1(reg.predict(test.values))


In [28]:
# sub = pd.DataFrame()
# sub['gift_id'] = test_ID
# sub['price'] = reg_pred
# sub.to_csv('submissionstacking1.csv',index=False)

In [29]:
def rmsle(y, y_pred):
    return np.log(mean_absolute_error(y, y_pred))

In [292]:
# stacked_averaged_models.fit(train.values, y_train)
# stacked_train_pred = stacked_averaged_models.predict(train.values)
# stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
# print(rmsle(y_train, stacked_train_pred))

In [31]:
model_rf.fit(train, np.log1p(y_train))
model_rf_train_pred = np.expm1(model_rf.predict(train))



NameError: name 'test' is not defined

In [33]:
model_rf_pred = np.expm1(model_rf.predict(test))
print(rmsle(y_train, model_rf_train_pred))

8.411166911223404


3.885730865544163

In [34]:
model_lgb.fit(train, np.log1p(y_train))
model_lgb_train_pred = np.expm1(model_lgb.predict(train))

model_lgb_pred = np.expm1(model_lgb.predict(test))
print(rmsle(y_train, model_lgb_train_pred))

8.05130085664147


6.010385321640361

In [35]:
GBoost.fit(train, np.log1p(y_train))
GBoost_train_pred = np.expm1(GBoost.predict(train))

GBoost_pred = np.expm1(GBoost.predict(test))
print(rmsle(y_train, model_lgb_train_pred))

8.05130085664147


6.010385321640361

In [36]:
model_avg = AveragingModels(models = ( GBoost,model_rf))

In [41]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=500,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_xgb.fit(train, np.log1p(y_train))
model_xgb_train_pred = np.expm1(model_xgb.predict(train))

model_xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, model_xgb_train_pred))

8.661020579787317


In [68]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y_train, model_rf_train_pred*0.20 + model_xgb_train_pred*0.20 + 0.70*model_lgb_train_pred )) #model_lgb_train_pred

RMSLE score on train data:
8.14666098807724


In [160]:
#model_avg_pred*0.10 + model_rf_pred*0.60 +

In [73]:
ensemble =   model_rf_pred*0.10 + model_lgb_pred*0.70 + model_xgb_pred*0.20

In [74]:
sub = pd.read_excel('../MachineHackElectronicPrice/Sample_Submission.xlsx')

In [75]:
sub['Price'] = ensemble

In [76]:
sub.to_excel('log2.xlsx')

In [328]:
# sub = pd.DataFrame()
# sub['gift_id'] = test_ID
# sub['price'] = ensemble
# sub.to_csv('submissionEnsemble.csv',index=False)